## Integration of Spatial Patterns and Machine Learning for Cholera Risk Assessment in Southern Ethiopia (PART 2)

### Abstract

This notebook builds upon the exploratory analysis of cholera case data from the Gedeo Zone, Ethiopia, during 2023, advancing towards predictive modeling of cholera severity based on clinical and epidemiological features. Leveraging individual-level data enriched by prior feature engineering, this phase applies machine learning techniques to develop, tune, and validate models that classify cholera severity, primarily informed by dehydration status. By integrating model selection, hyperparameter optimization, and performance evaluation, this notebook aims to deliver robust predictive tools to assist healthcare providers in the early identification of severe cases. These models have the potential to enhance clinical decision-making, improve patient outcomes, and optimize resource allocation in cholera-affected areas. The study concludes with an evaluation of the model interpretability, which aligns with the foundational insights established in the preceding exploratory data analysis notebook. Note that I will head straight to the model development.

## Data Description

In this notebook, I will use the individual-level cholera records that captured the cholera outbreak in Gedeo Zone, Ethiopia, during 2023. This dataset was sourced from [[1]](https://data.mendeley.com/datasets/7zz3tp5kt5/1).

---

### CHOLERA1: Individual-level cholera case records

This dataset contains detailed case-based information for cholera patients, including:

| Column                       | Description / Use                                                                                      |
| ---------------------------- | ---------------------------------------------------------------------------------------------------- |
| `SerialNo`                   | Unique identifier for each patient record                                                            |
| `AgeYear`                   | Patient age in years                                                                                   |
| `Kebelename`                | Name of kebele (the smallest administrative unit in Ethiopia) where the patient resides               |
| `HealthFacilityCTC`         | Health facility where case was seen                                                                   |
| `DateseenathealthfacilityM` | Date patient was seen/admitted at the health facility                                                 |
| `DateofonsetofdiseaseMMD`   | Date of symptom onset                                                                                 |
| `Dischargedate`             | Date of discharge from care                                                                           |
| `Dehaydration_status`       | Dehydration severity (none, some, severe) – key clinical indicator                                    |
| `Hospitalized`              | Whether patient was hospitalized (yes/no)                                                             |
| `sex`                      | Patient sex                                                                                          |
| `Region`, `Zone`, `District`| Administrative divisions for geographic context                                                     |
| `symptom`, `diarrhoea`, `vomit` | Presence of key symptoms                                                                          |
| `Occupation`                | Patient occupation                                                                                    |
| `water`, `Water_source`     | Source of drinking water                                                                              |
| `Contact_history`, `Travel_history` | Risk factor indicators: recent contact or travel                                             |
| `outcome`                  | Case outcome status (recovered, died, referred, etc.)                                                 |
| `Vaccination`              | Cholera vaccination status (yes/no/unknown)                                                          |
| `Latrine_avalability`      | Whether latrine was available at patient’s household                                                  |

### Data structure takeaways


- CHOLERA1 requires cleaning for text standardization and missing values.
- Temporal data present in CHOLERA1 (dates of onset, admission, discharge) allow temporal trend analysis.
- Clinical variables such as `Dehaydration_status` are critical for disease severity classification.
- Water and sanitation variables provide insight into possible environmental drivers and risk exposures.
- Some column names and elements may have been misspelled (e.g., `Dehaydration_status`), but these were retained in this notebook.


In building the model, I limited the data fields to the following: `Dehaydration_status`,
`DateofonsetofdiseaseMMD`, `water`, `Latrine_avalability`, `DateseenathealthfacilityM`. Here, I will transform `Dehaydration_status` into a cholera severity metric (also the target binary variable). Columns `DateofonsetofdiseaseMMD` and `DateseenathealthfacilityM` will be converted into a variable that I call onset-to-admission interval (which will then be further cleansed and clipped); this determines the number of days between when the symptom started and when the patient sought care. Finally, `water` and `Latrine_avalability` will be respectively transformed into one-hot encoded and single binary numeric features.

In [18]:
# LOAD DATASETS

import pandas as pd
import numpy as np
fname = 'Final data.csv'
cols_needed = ['Dehaydration_status','DateofonsetofdiseaseMMD', 'water', 'Latrine_avalability', 'DateseenathealthfacilityM']
df_cholera = pd.read_csv(fname, usecols=cols_needed)

In [19]:
# Display the unique elements per column in df_cholera
unique_elements = {col: df_cholera[col].unique() for col in df_cholera.columns}
print(unique_elements)

{'DateseenathealthfacilityM': array([' 7/30/2023', '  9/8/2023', ' 8/14/2023', '  6/9/2023',
       ' 5/12/2023', ' 9/21/2023', ' 5/13/2023', '  8/5/2023',
       '  7/9/2023', ' 8/13/2023', '10/19/2023', ' 9/20/2023',
       ' 9/17/2023', ' 5/25/2023', '10/25/2023', ' 10/9/2023',
       ' 9/15/2023', ' 9/23/2023', ' 8/17/2023', ' 9/26/2023',
       ' 7/17/2023', ' 10/5/2023', ' 6/10/2023', ' 5/17/2023',
       '11/12/2023', ' 9/22/2023', ' 7/11/2023', ' 7/23/2023',
       ' 9/24/2023', ' 7/28/2023', '  7/8/2023', '  8/1/2023',
       '10/20/2023', ' 4/22/2023', '10/16/2023', ' 11/2/2023',
       ' 7/27/2023', ' 9/25/2023', ' 10/2/2023', ' 7/20/2023',
       ' 9/29/2023', ' 7/24/2023', ' 7/13/2023', ' 4/21/2023',
       ' 9/28/2023', ' 5/28/2023', ' 7/14/2023', ' 7/19/2023',
       ' 8/18/2023', ' 5/21/2023', '11/10/2023', ' 7/29/2023',
       '  8/6/2023', ' 6/27/2023', ' 8/16/2023', ' 4/29/2023',
       '10/14/2023', ' 8/11/2023', '  7/7/2023', '10/28/2023',
       '  8/2/2023', ' 10

In [20]:
# Cleaning includes:
# - Date parsing and cleaning
# - String standardization (lowercase, strip)
# - Unification of categories (for `water`)
# - Missing value handling
# - Creating a binary target variable called `cholera_severe` based on severe dehydration status 

# ========== Step 1: Clean Date Columns ==========
# Strip and convert date columns to datetime, coercing errors to NaT
for col in ['DateseenathealthfacilityM']:
    df_cholera[col] = pd.to_datetime(df_cholera[col].astype(str).str.strip(), errors='coerce')

# Confirm existing DateofonsetofdiseaseMMD is datetime type, if not convert
if not pd.api.types.is_datetime64_any_dtype(df_cholera['DateofonsetofdiseaseMMD']):
    df_cholera['DateofonsetofdiseaseMMD'] = pd.to_datetime(df_cholera['DateofonsetofdiseaseMMD'], errors='coerce')

# ========== Step 2: Standardize Text Columns ==========
text_cols = ['water', 'Latrine_avalability']

for col in text_cols:
    df_cholera[col] = df_cholera[col].astype(str).str.lower().str.strip()

# ========== Step 3: Unify Water Source Categories ==========
# Helper function to unify water categories
def unify_water_source(x):
    if pd.isna(x) or x == 'nan' or x == 'unknown':
        return 'unknown'
    x = x.lower().strip()
    if 'river' in x:
        return 'river'
    elif 'protected spring' in x or 'protected spiring' in x:
        return 'protected spring'
    elif 'unprotected spring' in x:
        return 'unprotected spring'
    elif 'hand pump' in x:
        return 'hand pump'
    elif 'hand dug well' in x:
        return 'hand dug well'
    elif 'bore hole' in x:
        return 'bore hole water'
    elif 'rain water' in x:
        return 'rain water'
    elif 'tap water' in x:
        return 'tap water'
    else:
        return x

# Apply to both water columns
df_cholera['water'] = df_cholera['water'].apply(unify_water_source)

# ========== Step 4: Handle Missing Values for Categorical Columns ==========
fill_unknown_cols = ['Latrine_avalability']

for col in fill_unknown_cols:
    df_cholera[col] = df_cholera[col].replace(['nan', np.nan, 'none', ''], 'unknown')

# ========== Step 5: Create Target Variable ==========
# Example: binary for severe dehydration (1 if severe, else 0)
df_cholera['cholera_severe'] = df_cholera['Dehaydration_status'].apply(lambda x: 1 if 'severe' in str(x) else 0)


# ========== Step 6: Onset-to-admission interval ==========
df_cholera['days_onset_to_admission'] = (
    df_cholera['DateseenathealthfacilityM'] - df_cholera['DateofonsetofdiseaseMMD']
).dt.days

df_cholera['days_onset_to_admission'].value_counts()



# Identify rows where time difference is negative
neg_diff_mask = df_cholera['days_onset_to_admission'] < 0

print(f"Number of rows with negative intervals: {neg_diff_mask.sum()}")

# Drop these rows
df_cholera = df_cholera.loc[~neg_diff_mask]

print(f"Rows after dropping: {df_cholera.shape[0]}")


# Cap values at 30
df_cholera['days_onset_to_admission'] = df_cholera['days_onset_to_admission'].clip(upper=30)

df_cholera = df_cholera[df_cholera['days_onset_to_admission'].notnull() & (df_cholera['days_onset_to_admission'] >= 0)].copy()

# ========== Step 7: Final Check ==========
print("Sample cleaned data:")
print(df_cholera.head())

# Check unique counts of key columns after cleaning
for col in ['water', 'Latrine_avalability', 'Dehaydration_status', 'cholera_severe', 'days_onset_to_admission']:
    print(f"Unique values in {col}: {df_cholera[col].unique()}")

Number of rows with negative intervals: 3
Rows after dropping: 789
Sample cleaned data:
  DateseenathealthfacilityM  ... days_onset_to_admission
0                2023-07-30  ...                     0.0
1                2023-09-08  ...                     0.0
2                2023-08-14  ...                     0.0
3                2023-06-09  ...                     4.0
4                2023-05-12  ...                     1.0

[5 rows x 7 columns]
Unique values in water: ['tap water' 'river' 'protected spring' 'hand pump' 'unknown'
 'hand dug well' 'bore hole water']
Unique values in Latrine_avalability: ['yes' 'no' 'unknown']
Unique values in Dehaydration_status: ['severe dehaydration' 'some dehaydration' 'no dehaydration' 'Unknown']
Unique values in cholera_severe: [1 0]
Unique values in days_onset_to_admission: [ 0.  4.  1.  2. 30.  3.  5.]


# Modeling phase


The next code block now implements the full machine learning pipeline to predict cholera severity using epidemiological and clinical data. After preprocessing the primary dataset, it is then split into stratified training and testing subsets to maintain class proportions. A preprocessing pipeline is defined using `ColumnTransformer` to standardize the numeric onset-to-admission interval while passing one-hot encoded categorical features unchanged. To address class imbalance, the scale factor `scale_pos_weight` is computed for use by **XGBoost**. The code performs **Bayesian hyperparameter optimization** with 5-fold cross-validation on the preprocessed training set using the XGBoost `DMatrix` format, tuning parameters such as tree depth, gamma, subsample, and learning rate for improved ROC-AUC. After identifying the best hyperparameters, a full pipeline combining preprocessing and the tuned XGBoost model is trained with early stopping evaluated on the preprocessed test set to prevent overfitting. Model performance is assessed via classification metrics and ROC-AUC on unseen test data. For comparison, a similarly preprocessed Random Forest classifier is trained and evaluated as a baseline. The entire pipeline highlights robust feature engineering, data preprocessing, hyperparameter tuning, and model evaluation to build an effective predictive system for cholera severity classification.

In [21]:
!pip install bayesian-optimization
!pip install --upgrade xgboost

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, classification_report, accuracy_score, confusion_matrix

from xgboost import XGBClassifier, DMatrix, cv
from bayes_opt import BayesianOptimization
import xgboost as xgb

# ========== STEP 1: Define Features and Target with 'days_onset_to_admission' ==========

def prepare_features_target(df):
    """
    Prepare features X and target y including:
    - One-hot encoding of select water sources
    - Binary encoding of latrine availability
    - Numeric inclusion of days from onset to admission (capped and cleaned)
    """

    # Target
    y = df['cholera_severe']

    # Define categorical water sources of interest
    water_categories = ['tap water', 'river', 'protected spring', 'hand dug well', 'bore hole water']

    # Create 'water_cleaned' categorical to group uncommon waters as 'other'
    df['water_cleaned'] = df['water'].apply(lambda x: x if x in water_categories else 'other')

    # One-hot encode
    water_ohe = pd.get_dummies(df['water_cleaned'], prefix='water')

    # Latrine binary encoding (yes=1, else=0)
    latrine_bin = df['Latrine_avalability'].apply(lambda x: 1 if x == 'yes' else 0).rename('latrine_presence')

    # Combine all features (water dummies + latrine + numeric onset-admission days)
    X = pd.concat([water_ohe, latrine_bin, df['days_onset_to_admission']], axis=1)

    return X, y

# ========== STEP 2: Split Data Function ==========

def split_data(X, y, test_size=0.15, random_state=42):
    return train_test_split(X, y, stratify=y, test_size=test_size, random_state=random_state)

# ========== STEP 3: Evaluation ==========

def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    if hasattr(model, "predict_proba"):
        y_proba = model.predict_proba(X_test)[:,1]
    elif hasattr(model, "decision_function"):
        y_proba = model.decision_function(X_test)
    else:
        y_proba = None

    print("Classification Report:")
    print(classification_report(y_test, y_pred))

    # Calculate and print accuracy
    acc = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {acc:.4f}")

    if y_proba is not None:
        print(f"ROC AUC: {roc_auc_score(y_test, y_proba):.4f}")

    # Print confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    print("Confusion Matrix:")
    print(cm)
    


def get_scale_pos_weight(y):
    num_pos = np.sum(y == 1)
    num_neg = np.sum(y == 0)
    return num_neg / num_pos if num_pos > 0 else 1

# ========== STEP 4: Bayesian Optimization CV Function ==========

def xgb_cv(max_depth, gamma, subsample, colsample_bytree, min_child_weight, learning_rate):
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        'use_label_encoder': False,
        'tree_method': 'hist',
        'max_depth': int(max_depth),
        'gamma': gamma,
        'subsample': subsample,
        'colsample_bytree': colsample_bytree,
        'min_child_weight': int(min_child_weight),
        'learning_rate': learning_rate,
        'scale_pos_weight': scale_pos_weight
    }

    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=100,
        nfold=5,
        stratified=True,
        early_stopping_rounds=10,
        seed=42,
        metrics='auc',
        verbose_eval=False
    )

    return cv_results['test-auc-mean'].max()

# ========== PRIMARY SCRIPT ==========

# Check missing columns
required_cols = ['cholera_severe', 'water', 'Latrine_avalability', 'DateseenathealthfacilityM', 'DateofonsetofdiseaseMMD']
missing_cols = set(required_cols) - set(df_cholera.columns)
if missing_cols:
    raise ValueError(f"Missing columns in df_cholera needed: {missing_cols}")

print("Preparing features and target...")
X, y = prepare_features_target(df_cholera)

print(f"Features shape after preparation: {X.shape}")

# Split train/test
X_train, X_test, y_train, y_test = split_data(X, y)

print(f"Train size: {X_train.shape[0]}, Test size: {X_test.shape[0]}")
print(f"Train target distribution:\n{y_train.value_counts(normalize=True)}")

# Convert any boolean columns to int 
bool_cols_train = X_train.select_dtypes(include='bool').columns
for col in bool_cols_train:
    X_train[col] = X_train[col].astype(int)
bool_cols_test = X_test.select_dtypes(include='bool').columns
for col in bool_cols_test:
    X_test[col] = X_test[col].astype(int)

# Calculate scale_pos_weight for XGBoost imbalance handling
scale_pos_weight = get_scale_pos_weight(y_train)
print(f"Calculated scale_pos_weight: {scale_pos_weight:.3f}")

# Preprocessing pipeline: scale only the numeric 'days_onset_to_admission'
numeric_features = ['days_onset_to_admission']
categorical_features = [col for col in X.columns if col != 'days_onset_to_admission']

preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), numeric_features),
    ('cat', 'passthrough', categorical_features)
])

# Fit preprocessor on train and transform both train and test for Bayesian Optimization DMatrix
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)


# Create DMatrix for Bayesian Optimization
dtrain = DMatrix(X_train_preprocessed, label=y_train)

# ========== Bayesian Optimization of XGBoost Hyperparameters ==========

pbounds = {
    'max_depth': (3, 10),
    'gamma': (0, 5),
    'subsample': (0.6, 1),
    'colsample_bytree': (0.6, 1),
    'min_child_weight': (1, 10),
    'learning_rate': (0.01, 0.3)
}

optimizer = BayesianOptimization(f=xgb_cv, pbounds=pbounds, random_state=42, verbose=2)

print("\nStarting Bayesian Optimization...\n")
optimizer.maximize(init_points=10, n_iter=25)

best_params = optimizer.max['params']
best_params['max_depth'] = int(best_params['max_depth'])
best_params['min_child_weight'] = int(best_params['min_child_weight'])

# Fix remaining params
best_params.update({
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'use_label_encoder': False,
    'tree_method': 'hist',
    'scale_pos_weight': scale_pos_weight,
    'seed': 1
})

print("\nBest hyperparameters found:")
print(best_params)

# ========== Final Model Training with Pipeline ==========



pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(**best_params, n_estimators=1000, early_stopping_rounds=20))
])


X_test_preprocessed = pipeline.named_steps['preprocessor'].transform(X_test)


### pass this preprocessed data as the eval_set inside `.fit()`:

pipeline.fit(
    X_train,
    y_train,
    classifier__eval_set=[(X_test_preprocessed, y_test)],
    classifier__verbose=20
)


print("\nXGBoost Performance on Test Set:")
evaluate_model(pipeline, X_test, y_test)

# ========== Random Forest for Comparison ==========

rf_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(
        n_estimators=500,
        class_weight='balanced',
        random_state=42,
        n_jobs=-1))
])

rf_pipeline.fit(X_train, y_train)

print("\nRandom Forest Performance on Test Set:")
evaluate_model(rf_pipeline, X_test, y_test)

print("\nModeling complete.")


Preparing features and target...
Features shape after preparation: (691, 8)
Train size: 587, Test size: 104
Train target distribution:
cholera_severe
0    0.630324
1    0.369676
Name: proportion, dtype: float64
Calculated scale_pos_weight: 1.705

Starting Bayesian Optimization...

|   iter    |  target   | max_depth |   gamma   | subsample | colsam... | min_ch... | learni... |
-------------------------------------------------------------------------------------------------
| 1         | 0.5116000 | 5.6217808 | 4.7535715 | 0.8927975 | 0.8394633 | 2.4041677 | 0.0552384 |
| 2         | 0.5158776 | 3.4065852 | 4.3308807 | 0.8404460 | 0.8832290 | 1.1852604 | 0.2912738 |
| 3         | 0.5448524 | 8.8270984 | 1.0616955 | 0.6727299 | 0.6733618 | 3.7381801 | 0.1621793 |
| 4         | 0.5305253 | 6.0236151 | 1.4561457 | 0.8447411 | 0.6557975 | 3.6293018 | 0.1162449 |
| 5         | 0.5224772 | 6.1924898 | 3.9258798 | 0.6798695 | 0.8056937 | 6.3317311 | 0.0234706 |
| 6         | 0.5474122 | 7.2528

# Results and discussion

A predictive pipeline was developed and evaluated using a dataset of 691 samples with eight engineered features, which were split into 587 training and 104 test instances (~85:15 train-test split). The target variable, `cholera_severity` (1 for severe, 0 otherwise), exhibited moderate class imbalance, with approximately 37% of cases being severe, which was addressed through a calculated `scale_pos_weight` of 1.71 in the XGBoost model. Bayesian optimization identified an optimal XGBoost model configuration with a deeper tree depth of 9, moderate regularization (gamma ≈ 0.30), reduced subsampling (0.6), 60% column subsampling, a minimum child weight of 3, and a moderate learning rate of 0.15. On the test set, the tuned XGBoost model achieved an accuracy of 67%, with balanced precision and recall of approximately 74% for the non-severe class and improved sensitivity and precision for the severe class at 55%, yielding an F1-score of 0.55. The overall ROC AUC improved to 0.71, indicating a substantially better discriminatory ability than that of the previous iteration. As a baseline comparison, the Random Forest model obtained slightly lower accuracy (66%) and ROC AUC (0.67), with a similar precision but somewhat lower recall for severe cases. These results underscore the potential of machine learning models, particularly XGBoost tuned via Bayesian optimization, to predict cholera severity using epidemiological and clinical data. Nevertheless, the moderate sensitivity observed in severe cases underscores the persistent challenges in optimizing class-specific performance. This suggests the need for further feature enhancement, improved preprocessing, or alternative modeling strategies to enhance clinical applicability and patient management.

# Concluding Remarks

This notebook presents a comprehensive machine learning workflow for predicting cholera severity using individual-level epidemiological and clinical data. By incorporating thoughtful feature engineering, including key temporal and environmental factors, and applying robust modeling techniques such as Bayesian-optimized XGBoost and Random Forest, the models achieved moderate predictive performance with respective ROC AUC scores of approximately 0.67 and 0.71. While these results demonstrate the promising potential for identifying high-risk cholera cases, the sensitivity to severe outcomes remains moderate, highlighting the ongoing need for richer datasets and further methodological advancements. Future work should prioritize the integration of additional clinical indicators, exploration of alternative and ensemble algorithms, and validation across external cohorts to strengthen model robustness and generalizability. Overall, this study advances the application of data-driven approaches to enhance timely clinical decision-making and support targeted public health interventions in cholera-affected settings. If you have questions, kindly send them in at jprmaulion[at]gmail[dot]com.